# API Quest
## Oslo

# HYPOTHESIS
- Rich countries have more Nobel Prizes
    - Nobel prize winners immigrate towards rich countries
    - Nobel prize winners immigrate towards stable countries
- Countries of birth / early education have more impact than countries of higher education
- Nobel Prizes Laureates are getting younger
- Nobel Prizes are awarded more to international teams than before

- Gender Differences: Is there a significant difference in the gender ratio among Nobel Prize winners? Has this changed over time?
- Geographic Distribution: In which countries or regions are Nobel Prize winners predominantly located? Has this distribution changed over time?
- Age of Winners: What is the age distribution of Nobel Prize winners? Are there any noticeable trends in age?
- Publications: Are there specific journals where Nobel Prize winners’ research is commonly published? How influential are these journals?

## HYPOTHESIS 1
- Men are over represented in Nobel Prizes

# DATA SOURCES

1. **Nobel Laureates Data**
	- **Nobel Prize Official Data**
	  - Description: Comprehensive information on all Nobel laureates, including their age, nationality, affiliation, prize category, and motivation.
	  - Link: [Nobel Prize Official Website](https://www.nobelprize.org/organization/developer-zone-2/)
	  - API: [Nobel Prize API](https://www.nobelprize.org/organization/developer-zone-2/)
	- **Kaggle Nobel Laureates Dataset**
	  - Description: A dataset compiled from the Nobel Prize official data, available in CSV format for easy analysis.
	  - Link: [Kaggle Nobel Prize Dataset](https://www.kaggle.com/datasets/imdevskp/nobel-prize/data)

3. **Economic Indicators**
	- **World Bank GDP Data**
	  - Description: GDP per capita and other economic indicators for countries worldwide.
	  - Link: [World Bank GDP per Capita](https://data.worldbank.org/indicator/NY.GDP.PCAP.CD)
	- **Heritage Foundation Index of Economic Freedom**
	  - Description: Measures economic freedom in countries across 12 quantitative and qualitative factors.
	  - Link: [Index of Economic Freedom](https://www.heritage.org/index/)

4. **Education Expenditure and Statistics**
	- **UNESCO Education Data**
	  - Description: Data on government expenditure on education as a percentage of GDP and total government expenditure.
	  - Link: [UNESCO Education Expenditure](http://data.uis.unesco.org/)
	- **OECD Education Statistics**
	  - Description: Detailed statistics on education spending, enrollment rates, and educational attainment among OECD countries.
	  - Link: [OECD Education at a Glance](https://www.oecd.org/education/education-at-a-glance/)

5. **Gender Statistics**
	- **UNESCO Gender Parity Index**
	  - Description: Data on gender parity in education and literacy rates.
	  - Link: [UNESCO Gender Equality Data](http://data.uis.unesco.org/)
	- **World Bank Gender Data Portal**
	  - Description: Comprehensive data on gender equality indicators globally.
	  - Link: [World Bank Gender Data](https://datatopics.worldbank.org/gender/)

18. **Demographic and Socioeconomic Data**
	 - **United Nations Educational, Scientific and Cultural Organization (UNESCO) Institute for Statistics**
		- Description: Data on education, literacy rates, and demographic factors.
		- Link: [UNESCO UIS Data](http://data.uis.unesco.org/)
	 - **OECD Social and Welfare Statistics**
		- Description: Indicators on social protection, income inequality, and more.
		- Link: [OECD Social Data](https://www.oecd.org/social/soc/)



## Selected data sources

1. Nobel API
2. https://uis.unesco.org/
3. https://databank.worldbank.org/source/world-development-indicators

In [266]:
#imports
import os
import json
import requests
import pandas as pd
from dotenv import load_dotenv
import seaborn as sb

In [267]:
#settings
pd.set_option('display.max_colwidth', 900)

In [268]:
#load env
load_dotenv()
token = os.getenv('TOKEN')
print(token)

test


In [269]:
#TODO: Get the data from the API
enrollment_df = pd.read_csv('sources/school_enrolment_gender.csv')
enrollment_df

,Country Name,Country Code,Series Name,Series Code,1974 [YR1974],1975 [YR1975],1976 [YR1976],1977 [YR1977],1978 [YR1978],1979 [YR1979],...,2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022],2023 [YR2023]
0,United States,USA,"School enrollment, tertiary, male (% gross)",SE.TER.ENRR.MA,54.3086318969727,57.4090118408203,60.9704818725586,56.530689239502,55.6341514587402,53.5002593994141,...,75.3973617553711,75.7590179443359,76.0622711181641,74.9971923828125,74.8950500488281,73.9934616088867,72.8402481079102,68.7112731933594,64.5381774902344,..
1,United States,USA,"School enrollment, tertiary, female (% gross)",SE.TER.ENRR.FE,43.1207695007324,47.2997817993164,50.4750900268555,51.2817993164062,53.4821510314941,53.9854583740234,...,102.599388122559,102.747329711914,102.293800354004,102.011329650879,102.354042053223,102.425331115723,102.956741333008,101.710220336914,94.8502197265625,..
2,United States,USA,"School enrollment, secondary, male (% gross)",SE.SEC.ENRR.MA,..,..,..,..,87.7746810913086,..,...,96.3625335693359,97.1537170410156,99.0528335571289,99.1985168457031,99.7356567382812,100.902122497559,101.560943603516,101.046821594238,96.351806640625,..
3,United States,USA,"School enrollment, secondary, female (% gross)",SE.SEC.ENRR.FE,..,..,..,..,88.334831237793,..,...,97.5130462646484,98.1735763549805,98.4747085571289,98.695556640625,98.7960586547852,99.189453125,99.4142990112305,101.344367980957,98.6511611938477,..
4,United States,USA,"School enrollment, primary, male (% gross)",SE.PRM.ENRR.MA,..,..,..,..,..,..,...,99.8374404907227,100.312042236328,101.445777893066,102.204460144043,101.314239501953,100.634078979492,100.546607971191,98.5913619995117,96.9589233398438,..
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,Data from database: World Development Indicators,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [270]:
url= "https://api.nobelprize.org/2.1/laureates"
response = requests.get(url)
data = response.json()
laureate_infos = {}
for laureate in data['laureates']:
    laureate_infos[laureate['id']]= {
        'Name': laureate['knownName']['en'],
        'Gender': laureate.get('gender',None),
        'Country': laureate['birth']['place']['country']['en'] if 'birth' in laureate and 'place' in laureate['birth'] and 'country' in
        laureate['birth']['place'] else 'None',
        'Prize_year': laureate['nobelPrizes'][0]['awardYear'],
        'Prize_category': laureate['nobelPrizes'][0]['category']['en'],}
        #'Prize_affiliations': ['nobelPrizes'][0]['affiliations'][0]['name']['en']}
print(laureate_infos)
df = pd.DataFrame.from_dict(laureate_infos,orient ='index')
print(df)

{'745': {'Name': 'A. Michael Spence', 'Gender': 'male', 'Country': 'USA', 'Prize_year': '2001', 'Prize_category': 'Economic Sciences'}, '102': {'Name': 'Aage N. Bohr', 'Gender': 'male', 'Country': 'Denmark', 'Prize_year': '1975', 'Prize_category': 'Physics'}, '779': {'Name': 'Aaron Ciechanover', 'Gender': 'male', 'Country': 'British Protectorate of Palestine', 'Prize_year': '2004', 'Prize_category': 'Chemistry'}, '259': {'Name': 'Aaron Klug', 'Gender': 'male', 'Country': 'Lithuania', 'Prize_year': '1982', 'Prize_category': 'Chemistry'}, '1004': {'Name': 'Abdulrazak Gurnah', 'Gender': 'male', 'Country': 'None', 'Prize_year': '2021', 'Prize_category': 'Literature'}, '114': {'Name': 'Abdus Salam', 'Gender': 'male', 'Country': 'India', 'Prize_year': '1979', 'Prize_category': 'Physics'}, '982': {'Name': 'Abhijit Banerjee', 'Gender': 'male', 'Country': 'India', 'Prize_year': '2019', 'Prize_category': 'Economic Sciences'}, '981': {'Name': 'Abiy Ahmed Ali', 'Gender': 'male', 'Country': 'Ethiop

In [271]:
raw_data = response.json()
nobels = pd.json_normalize(raw_data['laureates'])
display (nobels.head())


,id,fileName,gender,sameAs,links,nobelPrizes,knownName.en,knownName.se,givenName.en,givenName.se,...,death.place.countryNow.se,death.place.countryNow.sameAs,death.place.countryNow.latitude,death.place.countryNow.longitude,death.place.continent.en,death.place.continent.no,death.place.continent.se,death.place.locationString.en,death.place.locationString.no,death.place.locationString.se
0,745,spence,male,"[https://www.wikidata.org/wiki/Q157245, https://en.wikipedia.org/wiki/Michael_Spence]","[{'rel': 'laureate', 'href': 'https://api.nobelprize.org/2/laureate/745', 'action': 'GET', 'types': 'application/json'}, {'rel': 'external', 'href': 'https://www.nobelprize.org/laureate/745', 'title': 'A. Michael Spence - Facts', 'action': 'GET', 'types': 'text/html', 'class': ['laureate facts']}]","[{'awardYear': '2001', 'category': {'en': 'Economic Sciences', 'no': 'Økonomi', 'se': 'Ekonomi'}, 'categoryFullName': {'en': 'The Sveriges Riksbank Prize in Economic Sciences in Memory of Alfred Nobel', 'no': 'Sveriges Riksbanks pris i økonomisk vitenskap til minne om Alfred Nobel', 'se': 'Sveriges Riksbanks pris i ekonomisk vetenskap till Alfred Nobels minne'}, 'sortOrder': '2', 'portion': '1/3', 'dateAwarded': '2001-10-10', 'prizeStatus': 'received', 'motivation': {'en': 'for their analyses of markets with asymmetric information', 'se': 'för deras analys av marknader med assymetrisk informations'}, 'prizeAmount': 10000000, 'prizeAmountAdjusted': 15114754, 'affiliations': [{'name': {'en': 'Stanford University', 'no': 'Stanford University', 'se': 'Stanford University'}, 'nameNow': {'en': 'Stanford University'}, 'city': {'en': 'Stanford, CA', 'no': 'Stanford, CA', 'se': 'Stanford, CA'...",A. Michael Spence,A. Michael Spence,A. Michael,A. Michael,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,102,bohr,male,"[https://www.wikidata.org/wiki/Q103854, https://en.wikipedia.org/wiki/Aage_Bohr]","[{'rel': 'laureate', 'href': 'https://api.nobelprize.org/2/laureate/102', 'action': 'GET', 'types': 'application/json'}, {'rel': 'external', 'href': 'https://www.nobelprize.org/laureate/102', 'title': 'Aage N. Bohr - Facts', 'action': 'GET', 'types': 'text/html', 'class': ['laureate facts']}]","[{'awardYear': '1975', 'category': {'en': 'Physics', 'no': 'Fysikk', 'se': 'Fysik'}, 'categoryFullName': {'en': 'The Nobel Prize in Physics', 'no': 'Nobelprisen i fysikk', 'se': 'Nobelpriset i fysik'}, 'sortOrder': '1', 'portion': '1/3', 'dateAwarded': '1975-10-17', 'prizeStatus': 'received', 'motivation': {'en': 'for the discovery of the connection between collective motion and particle motion in atomic nuclei and the development of the theory of the structure of the atomic nucleus based on this connection', 'se': 'för upptäckten av sambandet mellan kollektiva rörelser och partikelrörelser i atomkärnor, samt den därpå baserade utvecklingen av teorien för atomkärnans struktur'}, 'prizeAmount': 630000, 'prizeAmountAdjusted': 4184870, 'affiliations': [{'name': {'en': 'Niels Bohr Institute', 'no': 'Niels Bohr Institute', 'se': 'Niels Bohr Institute'}, 'nameNow': {'en': 'Niels Bohr Insti...",Aage N. Bohr,Aage N. Bohr,Aage N.,Aage N.,...,Danmark,[https://www.wikidata.org/wiki/Q35],56.000000,10.000000,Europe,Europa,Europa,"Copenhagen, Denmark","København, Danmark","Köpenhamn, Danmark"
2,779,ciechanover,male,"[https://www.wikidata.org/wiki/Q233205, https://en.wikipedia.org/wiki/Aaron_Ciechanover]","[{'rel': 'laureate', 'href': 'https://api.nobelprize.org/2/laureate/779', 'action': 'GET', 'types': 'application/json'}, {'rel': 'external', 'href': 'https://www.nobelprize.org/laureate/779', 'title': 'Aaron Ciechanover - Facts', 'action': 'GET', 'types': 'text/html', 'class': ['laureate facts']}]","[{'awardYear': '2004', 'category': {'en': 'Chemistry', 'no': 'Kjemi', 'se': 'Kemi'}, 'categoryFullName': {'en': 'The Nobel Prize in Chemistry', 'no': 'Nobelprisen i kjemi', 'se': 'Nobelpriset i kemi'}, 'sortOrder': '1', 'portion': '1/3', 'dateAwarded': '2004-10-06', 'prizeStatus': 'received', 

In [275]:
def flatten(dictionnary, prefix=''):
    flattened = pd.json_normalize(dictionnary)

    if prefix:
        flattened = flattened.add_prefix(prefix + '.')
    
    for column in flattened.columns:
        sample = flattened[column].iloc[0]

        if isinstance(sample, list) and len(sample) > 0 and isinstance(sample[0], dict):

            inner_dict = flattened[column].apply(
                lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)

            flattened = pd.concat([flattened, flatten(inner_dict, column)], axis=1)
            flattened.drop(column, axis=1, inplace=True)

    return flattened


flattened = flatten(raw_data['laureates'])

In [273]:

export_csv = flattened.to_csv(r'./sources/nobel_laureates.csv', index = None, header=True)